# Mountain Car

#### Descripcion 


+ **Espacio de observacion (Box(2,))**

+ Posicion: [-1.2, 0.6]
+ Velocidad: [-0.07, 0.07]


+ **Espacio de accion (Discrete(3))**

+ Izquierda: 0
+ No hacer nada: 1
+ Derecha: 2


El objetivo es llevar el coche hasta la posicion 0.5

Para cada paso temporal que el coche no este en esa posicion, la recompensa sera -1.

In [1]:
# primero test aleatorio del entorno

import gym

In [2]:
entorno=gym.make('MountainCar-v0')

In [3]:
for episodio in range(5):  # 5 episodios
    
    done=False
    
    observacion=entorno.reset()  # observacion
    
    recompensa_total=0.          # recompensa total en cada episodio
    
    paso=0                       # paso en cada episodio
     
    while not done:
        entorno.render()  # muestra el entorno
        
        accion=entorno.action_space.sample() # accion aleatoria, se cambiara por el agente
        
        siguiente_estado, recompensa, done, info=entorno.step(accion)
        
        recompensa_total+=recompensa
        
        paso+=1
        
        observacion=siguiente_estado
        
    print ('\nEpisodio #{} finalizado en {} pasos. Recompensa total:{}'.format(episodio+1, paso, recompensa_total))
    
entorno.close()


Episodio #1 finalizado en 200 pasos. Recompensa total:-200.0

Episodio #2 finalizado en 200 pasos. Recompensa total:-200.0

Episodio #3 finalizado en 200 pasos. Recompensa total:-200.0

Episodio #4 finalizado en 200 pasos. Recompensa total:-200.0

Episodio #5 finalizado en 200 pasos. Recompensa total:-200.0


### Implementacion del agente

Ahora vamos a crear un agente basado en el algoritmo Q-learning (Q viene de quality, la calidad de la accion tomada en un estado dado). Dicho algoritmo tiene la ecuacion:

$$Q_{nueva}(e_{t}, a_{t})=(1-\alpha)·Q(e_{t}, a_{t}) + \alpha·[r_{t} + \gamma·\max_{a}Q(e_{t+1}, a_{t+1})]$$


donde:
+ $e_{t}$ es el estado en el tiempo t
+ $a_{t}$ es la accion en el tiempo t
+ $\alpha$ es la tasa de aprendizaje $(0<\alpha \leq{1})$
+ $Q(e_{t}, a_{t})$ es el viejo valor de calidad
+ $[r_{t} + \gamma·\max_{a}Q(e_{t+1}, a_{t+1})]$ es el valor aprendido
+ $r_{t}$ es la recompensa recibida al pasar del estado $e_{t}$ al estado $e_{t+1}$
+ $\gamma$ es el factor de descuento $(0\leq \gamma \leq 1)$. Evalua las recompensas recibidas anteriormente con un valor mayor que las recibidas posteriormente, se puede interpretar como la probabilidad de tener exito (o sobrevivir) en cada paso temporal+ $\max_{a}Q(e_{t+1}, a_{t+1})$ es la estimacion del valor optimo futuro

In [4]:
# se crea la clase agente (basada en numpy)

import numpy as np

class Agente(object):
    
    def __init__(self, entorno):
        self.obs_dim=entorno.observation_space.shape              # dimension espacio observacion
        self.obs_sup=entorno.observation_space.high               # limite superior
        self.obs_inf=entorno.observation_space.low                # limite inferior
        self.obs_bins=NUM_BINS                                    # discretizacion de un espacio continuo, numero de bins
        self.ancho_bin=(self.obs_sup-self.obs_inf)/self.obs_bins  # ancho de cada parte de la discretizacion
        self.dim_accion=entorno.action_space.n                    # dimension espacio accion
        
        self.Q=np.zeros((self.obs_bins+1, 
                         self.obs_bins+1,
                         self.dim_accion))  # array para guardar los Q-valores
        
        self.alfa=ALFA        # tasa de aprendizaje           
        self.gamma=GAMMA      # factor de descuento
        self.epsilon=EPSILON  # prob para escoger accion 
        
        
    def discretiza(self, obs):  # discretiza un espacio continuo
        return tuple(((obs-self.obs_inf)/self.ancho_bin).astype(int))  # binning
        
        
    def accion(self, obs):   # realiza la accion del agente
        obs_discreta=self.discretiza(obs) # discretiza espacio
        
        # politica epsilon-greedy para escoger la accion (mejor probabilidad 1-eps, peor es epsilon)
        if self.epsilon>MIN_EPSILON: self.epsilon-=DECAY_EPSILON
            
        if np.random.random()>self.epsilon: return np.argmax(self.Q[obs_discreta])
        else: return np.random.choice([a for a in range(self.dim_accion)])
        
        
    def aprende(self, obs, acciona, recompensa, siguiente_obs): # metodo de aprendizaje
        obs_discreta=self.discretiza(obs)                       # discretiza las observaciones
        siguiente_obs_discreta=self.discretiza(siguiente_obs)
        
        td_objetivo=recompensa+self.gamma*np.max(self.Q[siguiente_obs_discreta]) # valor de aprendizaje
        td_error=td_objetivo-self.Q[obs_discreta][acciona]                       # error aprendizaje
        self.Q[obs_discreta][acciona]+=self.alfa*td_error                        # actualizacion Q-valores

#### Fase entrenamiento-testeo

In [5]:
# primero las constantes (se pueden considerar hiperparametros)

MIN_EPSILON=0.005                               # epsilon minimo, probabilidad minima para escoger una accion
EPSILON=1.                                      # prob para escoger accion
NUM_MAX_EPIS=50000                              # numero maximo de episodios
PASOS_POR_EPI=200                               # pasos por episodio
NUM_MAX_PASOS=NUM_MAX_EPIS*PASOS_POR_EPI        # numero maximo de pasos por episodio  
DECAY_EPSILON=500*MIN_EPSILON/NUM_MAX_PASOS     # ajuste del valor de epsilon 
ALFA=0.05                                       # tasa de aprendizaje
GAMMA=0.98                                      # factor de descuento
NUM_BINS=20                                     # numero de binnings para discretizacion

In [6]:
# funcion de entrenamiento

def train(agente, entorno):   
    mejor_recompensa=-float('inf')  # se inicializa la mejor recompensa (al minimo posible)
    
    for episodio in range(NUM_MAX_EPIS):
        done=False
        obs=entorno.reset()
        recompensa_total=0.
        
        while not done:
            accion=agente.accion(obs)
            siguiente_obs, recompensa, done, info=entorno.step(accion)
            agente.aprende(obs, accion, recompensa, siguiente_obs)
            obs=siguiente_obs
            recompensa_total+=recompensa
            
        if recompensa_total>mejor_recompensa: mejor_recompensa=recompensa_total
        if episodio%2000==0:
            print ('Episodio:{}..|..Recompensa:{}, Mejor Recompensa:{}..|..Epsilon:{}'.format(episodio, recompensa_total, mejor_recompensa, agente.epsilon))
    
    return np.argmax(agente.Q, axis=2)   # devuelve la politica

In [7]:
# funcion de testeo

def test(agente, entorno, politica):
    done=False
    obs=entorno.reset()
    recompensa_total=0.
    
    while not done:
        accion=politica[agente.discretiza(obs)]
        siguiente_obs, recompensa, done, info=entorno.step(accion)
        obs=siguiente_obs
        recompensa_total+=recompensa
        
    return recompensa_total

In [8]:
# se ejecuta todo

if __name__=='__main__':
    
    entorno=gym.make('MountainCar-v0')
    agente=Agente(entorno)
    politica=train(agente, entorno)
    
    gym_monitor='./gym_monitor'  # para guardar video
    entorno=gym.wrappers.Monitor(entorno, gym_monitor, force=True)
    
    for i in range(500):
        test(agente, entorno, politica)
        
    entorno.close()

Episodio:0..|..Recompensa:-200.0, Mejor Recompensa:-200.0..|..Epsilon:0.999949999999993
Episodio:2000..|..Recompensa:-200.0, Mejor Recompensa:-200.0..|..Epsilon:0.8999499999860152
Episodio:4000..|..Recompensa:-200.0, Mejor Recompensa:-200.0..|..Epsilon:0.7999499999720374
Episodio:6000..|..Recompensa:-200.0, Mejor Recompensa:-200.0..|..Epsilon:0.6999499999580596
Episodio:8000..|..Recompensa:-200.0, Mejor Recompensa:-168.0..|..Epsilon:0.5999579999440829
Episodio:10000..|..Recompensa:-200.0, Mejor Recompensa:-161.0..|..Epsilon:0.5000474999301177
Episodio:12000..|..Recompensa:-200.0, Mejor Recompensa:-153.0..|..Epsilon:0.40107799993824905
Episodio:14000..|..Recompensa:-200.0, Mejor Recompensa:-125.0..|..Epsilon:0.3046594999461811
Episodio:16000..|..Recompensa:-200.0, Mejor Recompensa:-118.0..|..Epsilon:0.2136659999496329
Episodio:18000..|..Recompensa:-157.0, Mejor Recompensa:-115.0..|..Epsilon:0.12848024994718332
Episodio:20000..|..Recompensa:-145.0, Mejor Recompensa:-109.0..|..Epsilon:0.0